```
embedder = OllamaEmbeddings()

vector = embedder.embed_documents([
    "hi", "how", "are", "you", "longer some word!"
])

for i in range(len(vector)):
    print(len(vector[i]))
```

In [1]:
from langchain.chat_models import ChatOllama
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

In [2]:
cache_dir = LocalFileStore("./cache/")


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
    # length_function=len,
)
loader = UnstructuredFileLoader("./files/Chaptor One.txt")

docs = loader.load_and_split(text_splitter=splitter)

In [3]:
ollama = ChatOllama(
    model = "llama2:7b",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [4]:
embedding = OllamaEmbeddings()


cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embedding,cache_dir)

vectorstore = Chroma.from_documents(docs, embedding)

In [ ]:
result = vectorstore.similarity_search("where does winston live?")

len(result)

4

In [ ]:
chain = RetrievalQA.from_chain_type(
    llm=ollama,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

In [ ]:
chain.run("Where does Winston live")

In [ ]:
chain.run("Describe Vectory Mension")